In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv('heart_disease_data.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [4]:
df.duplicated().sum()

1

In [5]:
df = df.drop_duplicates()

In [6]:
df.duplicated().sum()

0

In [7]:
missing_data = df.isnull().sum()
print("\nMissing data:")
print(missing_data)


Missing data:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [8]:
df.shape

(302, 14)

In [9]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,302.00000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,54.42053,0.682119,0.963576,131.602649,246.500000,0.149007,0.526490,149.569536,0.327815,1.043046,1.397351,0.718543,2.314570,0.543046
std,9.04797,0.466426,1.032044,17.563394,51.753489,0.356686,0.526027,22.903527,0.470196,1.161452,0.616274,1.006748,0.613026,0.498970
min,29.00000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.00000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.50000,1.000000,1.000000,130.000000,240.500000,0.000000,1.000000,152.500000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.00000,1.000000,2.000000,140.000000,274.750000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.00000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       302 non-null    int64  
 1   sex       302 non-null    int64  
 2   cp        302 non-null    int64  
 3   trestbps  302 non-null    int64  
 4   chol      302 non-null    int64  
 5   fbs       302 non-null    int64  
 6   restecg   302 non-null    int64  
 7   thalach   302 non-null    int64  
 8   exang     302 non-null    int64  
 9   oldpeak   302 non-null    float64
 10  slope     302 non-null    int64  
 11  ca        302 non-null    int64  
 12  thal      302 non-null    int64  
 13  target    302 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 35.4 KB


In [11]:
df.target.value_counts()
# 1 represents positive case of heart disease
# 2 represents negetive case of heart disease

target
1    164
0    138
Name: count, dtype: int64

In [12]:
x = df.drop(columns = ['target'])
y = df['target']

In [13]:
x

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size = 0.2, stratify = y, random_state = 4)

# Model Training

In [16]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression ()
model.fit (x_train, y_train)

from sklearn.linear_model import LinearRegression
lr = LinearRegression ()
lr.fit (x_train, y_train)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier ()
rf.fit (x_train, y_train)

from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier ()
gbc.fit (x_train, y_train)

from xgboost import XGBClassifier
xg = XGBClassifier ()
xg.fit (x_train, y_train)

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit (x_train, y_train)


DecisionTreeClassifier()

In [17]:
print('Linear Regression Train Score is', lr.score(x_train, y_train) * 100)
print('Random Forest Classifier Train Score is', rf.score(x_train, y_train) * 100)
print('Gradient Boosting Regression Train Score is', gbc.score(x_train, y_train) * 100)
print('XGBoost Regression Train Score is', xg.score(x_train, y_train) * 100)
print('Decision Tree Regression Train Score is', tree.score(x_train, y_train) * 100)

Linear Regression Train Score is 50.610520427894755
Random Forest Classifier Train Score is 100.0
Gradient Boosting Regression Train Score is 99.5850622406639
XGBoost Regression Train Score is 100.0
Decision Tree Regression Train Score is 100.0


In [18]:
print('Linear Regression Test Score is', lr.score(x_test, y_test) * 100)
print('Random Forest ClAassifier Test Score is', rf.score(x_test, y_test) * 100)
print('Gradient Boosting Classifier Test Score is', gbc.score(x_test, y_test) * 100)
print('XGBoost Classifier Test Score is', xg.score(x_test, y_test) * 100)
print('Decision Tree Classifier Test Score is', tree.score(x_test, y_test) * 100)

Linear Regression Test Score is 55.04787063402162
Random Forest ClAassifier Test Score is 85.24590163934425
Gradient Boosting Classifier Test Score is 80.32786885245902
XGBoost Classifier Test Score is 85.24590163934425
Decision Tree Classifier Test Score is 80.32786885245902


In [19]:
#Accuracy on training data
train_score = model.score (x_train, y_train)

print ('Accuracy on test data', train_score)


Accuracy on test data 0.8423236514522822


In [20]:
xg.fit(x,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [21]:
y_pred = lr.predict (x_test)
y_pred2 = rf.predict (x_test)
y_pred3 = gbc.predict (x_test)
y_pred4 = xg.predict (x_test)
y_pred5 = tree.predict (x_test)

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [24]:
y_pred

array([ 0.83565633,  0.69959862,  0.85532337,  0.31540623, -0.10452801,
        0.8489513 ,  0.55662573,  0.20899343,  0.65977112,  1.14684871,
        1.07487668,  0.75223206,  0.63912967,  0.7248282 ,  0.79740958,
       -0.15060102,  0.2666636 ,  0.88843619, -0.06174497,  0.32504814,
        0.59801415,  1.19984518,  0.17117181,  0.05662897,  0.6772833 ,
        0.49975806,  0.92984414,  0.83849676,  0.71275026,  0.00341806,
       -0.0230804 ,  0.39716237,  0.67120448, -0.03288201,  0.37828012,
        0.78084423,  0.12806175, -0.20872014,  0.67914137, -0.21073576,
        0.34860359,  0.64488549,  0.97401529,  0.96817659,  0.85166672,
        0.67792202,  0.98306723,  0.72418599,  0.54697586,  1.19140481,
        0.86173849,  0.95896848,  0.60920348,  0.37387878,  0.20300125,
        0.17237272,  0.17991975,  0.70547426,  0.64964631,  0.83884802,
        0.79735092])

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [26]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

Mean Absolute Error: 0.2684040717719824
Mean Squared Error: 0.11162528227402319
Root Mean Squared Error: 0.3341036998807753
R-squared: 0.5504787063402162


In [27]:
input_data = (37,1,2,130,250,0,1,187,0,3.5,0,0,2)

# Change the input to numpy array
data = np.array (input_data)

input_data_reshaped = data.reshape(1, -1)

prediction = xg.predict (input_data_reshaped)
print(prediction)

if (prediction == 0):
    print('This person Dosent have Heart Disease')
else:
    print('This Person Has Heart Disease')

[1]
This Person Has Heart Disease


In [28]:
input_data = (37,1,2,130,250,0,1,187,0,3.5,0,0,2)

# Change the input to numpy array
data = np.array (input_data)

input_data_reshaped = data.reshape(1, -1)

prediction = model.predict (input_data_reshaped)
print(prediction)

if (prediction == 0):
    print('This person Dosent have Heart Disease')
else:
    print('This Person Has Heart Disease')

[1]
This Person Has Heart Disease


In [29]:

input_data = (58,1,1,120,284,0,0,160,0,1.8,1,0,2)

# Change the input to numpy array
data = np.array (input_data)

input_data_reshaped = data.reshape(1, -1)  #

prediction = model.predict (input_data_reshaped)
print(prediction)

if (prediction == 0):
    print('This person Dosent have Heart Disease')
else:
    print('This Person Has Heart Disease')

[1]
This Person Has Heart Disease


In [30]:
input_data = (58,1,2,112,230,0,0,165,0,2.5,1,1,3)

# Change the input to numpy array
data = np.array (input_data)

input_data_reshaped = data.reshape(1, -1)  #

prediction = model.predict (input_data_reshaped)
print(prediction)

if (prediction == 0):
    print('This person Does not have Heart Disease')
else:
    print('This Person Definately Has Heart Disease')

[0]
This person Does not have Heart Disease


In [31]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [32]:
from tkinter import *
import joblib
import pandas as pd

def show_entry_fields():
    age = float(e1.get())
    sex = float(e2.get())
    cp = float(e3.get())
    trestbps = float(e4.get())
    chol = float(e5.get())
    fbs = float(e6.get())
    restecg = float(e7.get())
    thalach = float(e8.get())
    exang = float(e9.get())
    oldpeak = float(e10.get())
    slope = float(e11.get())
    ca = float(e12.get())
    thal = float(e13.get())

    model = joblib.load('Heart Disease')
    data_new = pd.DataFrame({
        'age': [age],
        'sex': [sex],
        'cp': [cp],
        'trestbps': [trestbps],
        'chol': [chol],
        'fbs': [fbs],
        'restecg': [restecg],
        'thalach': [thalach],
        'exang': [exang],
        'oldpeak': [oldpeak],
        'slope': [slope],
        'ca': [ca],
        'thal': [thal],
    })

    result = model.predict(data_new)
    Label(master, text="Heart Disease or not").grid(row=16)
    Label(master, text=result).grid(row=17)
    print("Car Purchase amount", result[0])

master = Tk()
master.title("Heart Disease Project")
label = Label(master, text="Property Price Prediction Using Machine Learning", bg="black", fg="white").grid(row=0, columnspan=2)

Label(master, text="age").grid(row=1)
Label(master, text="sex").grid(row=2)
Label(master, text="cp").grid(row=3)
Label(master, text="trestbps").grid(row=4)
Label(master, text="chol").grid(row=5)
Label(master, text="fbs").grid(row=6)
Label(master, text="restecg").grid(row=7)
Label(master, text="thalach").grid(row=8)
Label(master, text="exang").grid(row=9)
Label(master, text="oldpeak").grid(row=10)
Label(master, text="slope").grid(row=11)
Label(master, text="ca").grid(row=12)
Label(master, text="thal").grid(row=13)

e1 = Entry(master)
e2 = Entry(master)
e3 = Entry(master)
e4 = Entry(master)
e5 = Entry(master)
e6 = Entry(master)
e7 = Entry(master)
e8 = Entry(master)
e9 = Entry(master)
e10 = Entry(master)
e11 = Entry(master)
e12 = Entry(master)
e13 = Entry(master)

e1.grid(row=1, column=1)
e2.grid(row=2, column=1)
e3.grid(row=3, column=1)
e4.grid(row=4, column=1)
e5.grid(row=5, column=1)
e6.grid(row=6, column=1)
e7.grid(row=7, column=1)
e8.grid(row=8, column=1)
e9.grid(row=9, column=1)
e10.grid(row=10, column=1)
e11.grid(row=11, column=1)
e12.grid(row=12, column=1)
e13.grid(row=13, column=1)

Button(master, text='Predict', command=show_entry_fields).grid(row=15, columnspan=2)

mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Temp\ipykernel_29312\2136217775.py", line 6, in show_entry_fields
    age = float(e1.get())
          ^^^^^^^^^^^^^^^
ValueError: could not convert string to float: ''
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Temp\ipykernel_29312\2136217775.py", line 6, in show_entry_fields
    age = float(e1.get())
          ^^^^^^^^^^^^^^^
ValueError: could not convert string to float: ''
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^

In [33]:
round(model.intercept_,2)

TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
model.coef_

In [ ]:
pip install sweetviz
